In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv2DTranspose
from imageViz import ImageVisualizer
import os
import cv2 as cv
import numpy as np

In [2]:
conv_kwargs = {
    "padding"             : "SAME",
    "activation"          : keras.layers.LeakyReLU(alpha=0.2),
    "kernel_initializer"  : tf.random_normal_initializer(stddev=.1)
}

In [3]:
class Autoencoder(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.encoder = keras.Sequential([
             Conv2D(16, 8, 4, **conv_kwargs),
            Conv2D(16, 8, 4, **conv_kwargs),
            keras.layers.MaxPooling2D(),
            Conv2D(8, 4, 1, **conv_kwargs),
        ], name="ae_encoder")

        self.decoder = keras.Sequential([
        Conv2DTranspose(8, 4, 1, **conv_kwargs),
        Conv2DTranspose(16, 8, 4, **conv_kwargs),
        Conv2DTranspose(3, 8, 8, padding='same', kernel_initializer=tf.random_normal_initializer(stddev=.1))
    ], name='ae_decoder')

    def call(self, inputs):
        inputs = self.encoder(inputs)
        inputs = self.decoder(inputs)
        return inputs


In [4]:
def mse_bce_loss(y_true, y_pred):
    mse_loss = keras.losses.MeanSquaredError()     
    bce_loss = keras.losses.BinaryCrossentropy()
    mse = mse_loss(y_true, y_pred)
    bce = bce_loss(y_true, y_pred)
    loss = .8 * mse + .2 * bce
    return loss

In [5]:
ae_model = Autoencoder(name='autoencoder')

ae_model.build(input_shape = (1,256,256,3))   ## Required to see architecture summary
initial_weights = ae_model.get_weights() ## Just so we can reset out autoencoder

ae_model.summary()
ae_model.encoder.summary()
ae_model.decoder.summary()

ae_model.compile(
    optimizer   = keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss        = mse_bce_loss,
    metrics     = [
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.BinaryCrossentropy()
    ]
)

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ae_encoder (Sequential)     (1, 8, 8, 8)              21544     
                                                                 
 ae_decoder (Sequential)     (1, 256, 256, 3)          12315     
                                                                 
Total params: 33859 (132.26 KB)
Trainable params: 33859 (132.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "ae_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (1, 64, 64, 16)           3088      
                                                                 
 conv2d_1 (Conv2D)           (1, 16, 16, 16)           16400     
                                                               

2024-04-23 22:14:28.804104: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-04-23 22:14:28.804132: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-04-23 22:14:28.804136: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-04-23 22:14:28.804182: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-23 22:14:28.804205: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [16]:
data_dir = '../collapsed_data'
def get_data(sub_dir, size=(256,256)):
    dir = os.path.join(data_dir, sub_dir)
    files = os.listdir(dir)
    x = []
    y = []
    for f in files:
        path = os.path.join(dir, f)
        try:
            img = cv.imread(path)
            img = cv.resize(img, size)
            y.append(img)
        except:
            print(path)
            continue
    y = np.array(y, dtype=np.float32)
    y = y[:4000]
    x = np.copy(y)
    h,w = size
    rec_w = w//5
    # Make middle black
    x[:,:,rec_w*2:rec_w*3,:]=0
    y=y/255
    x = x/255
    # cv.imshow('x',x[21])
    # cv.waitKeyEx()
    # cv.imshow('y',y[21])
    # cv.waitKeyEx()
    print(x.shape)
    print(y.shape)
    print('done')
    return x,y


In [17]:
x_train,y_train = get_data('train')
#x_valid,y_valid = get_data('validation')

(4000, 256, 256, 3)
(4000, 256, 256, 3)
done


In [18]:
# Train the model
print('Fitting model')
#ae_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_valid,y_valid))
ae_model.fit(x_train, y_train, epochs=5, batch_size=64)
print('------------------------')

Fitting model
Epoch 1/5
63/63 [==============================] - 9s 122ms/step - loss: 0.1436 - mean_squared_error: 0.0311 - binary_crossentropy: 0.5939
Epoch 2/5
63/63 [==============================] - 7s 109ms/step - loss: 0.1429 - mean_squared_error: 0.0305 - binary_crossentropy: 0.5926
Epoch 3/5
63/63 [==============================] - 7s 112ms/step - loss: 0.1425 - mean_squared_error: 0.0302 - binary_crossentropy: 0.5917
Epoch 4/5
63/63 [==============================] - 7s 112ms/step - loss: 0.1420 - mean_squared_error: 0.0298 - binary_crossentropy: 0.5907
Epoch 5/5
63/63 [==============================] - 7s 111ms/step - loss: 0.1417 - mean_squared_error: 0.0296 - binary_crossentropy: 0.5902
------------------------


In [19]:
print('Evaluating model on testing data')
x_test,y_test = get_data('test')
ae_model.evaluate(x_test, y_test, batch_size=32)

Evaluating model on testing data
(500, 256, 256, 3)
(500, 256, 256, 3)
done
16/16 [==============================] - 1s 39ms/step - loss: 0.1446 - mean_squared_error: 0.0320 - binary_crossentropy: 0.5952


[0.1446211040019989, 0.031970277428627014, 0.5952244400978088]

In [20]:
x = x_test[:5]
pred = ae_model.predict(x)
for i in range(5):
    cv.imshow('im', x[i])
    cv.waitKeyEx()
    cv.imshow('pred', pred[i])
    cv.waitKeyEx()

1/1 [==============================] - 0s 30ms/step
